In [1]:
#!/usr/bin/env python
# coding: utf-8

import math
import random
import os
import json


import inspect
import code

import torch
from torch.nn.functional import cross_entropy, log_softmax

import datasets
from util import const, icl, lm, infogain, util

In [2]:
args = type('', (), {})()
# parser.add_argument('out_file', type=str)
# args.out_file = NotImplemented
# parser.add_argument('--n_tries', type=int, default=10)
args.n_tries = NotImplemented
# parser.add_argument('--p', type=float, default=0.5)
args.p = NotImplemented
# parser.add_argument('--gen_len', type=int, default=50)
args.gen_len = NotImplemented
# parser.add_argument('--model', type=str, default='n125m')
args.model = '2l'
# parser.add_argument('--n_demos', type=int, default=10)
args.n_demos = NotImplemented
# parser.add_argument('--n_gen_bs', type=int, default=10)
args.n_gen_bs = NotImplemented
# parser.add_argument('--device', type=str, default='cuda')
args.device = 'cuda'
# parser.add_argument('--seed', type=int, default=0)
args.seed = 0
# parser.add_argument('--worker_id', type=int, default=None)
args.worker_id = NotImplemented
# parser.add_argument('--n_shards', type=int, default=None)
args.n_shards = NotImplemented
# parser.add_argument('--print_int', type=int, default=1)
args.print_int = NotImplemented
# parser.add_argument('--override', action='store_true')
args.override = NotImplemented

# if args.override:
#     with open(args.out_file, 'w') as out:
#         pass
# else:
#     assert(not os.path.exists(args.out_file))

util.set_all_seeds(args.seed)

In [3]:
with open('data/multirc/dev.json') as f:
  data = json.load(f)['data']

In [17]:
idxs = set()
for datum in data:
  datum = datum['paragraph']
  passage_text = datum['text']
  for question in datum['questions']:
    question_text = question['question']
    idxs.update(question['sentences_used'])
    answers = question['answers']

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19}

In [14]:
import re
def proc_passage(s):
  s = re.sub('<b>.*?</b>', '', s)
  secs = s.split('<br>')
  secs = [ sec.strip() for sec in secs ]
  secs = list(filter(lambda s: len(s) > 0, secs))
  secs = [ ' '.join(sec.split()) for sec in secs]
  return secs

In [15]:
proc_passage(passage_text)

['Bin Laden began delivering diatribes against the United States before he left Saudi Arabia.',
 'He continued to do so after he arrived in Sudan.',
 'In early 1992, the al Qaeda leadership issued a fatwa calling for jihad against the Western "occupation" of Islamic lands.',
 "Specifically singling out U.S. forces for attack, the language resembled that which would appear in Bin Laden's public fatwa in August 1996.",
 'In ensuing weeks, Bin Laden delivered an often-repeated lecture on the need to cut off "the head of the snake."',
 'By this time, Bin Laden was well-known and a senior figure among Islamist extremists, especially those in Egypt, the Arabian Peninsula, and the Afghanistan-Pakistan border region.',
 'Still, he was just one among many diverse terrorist barons.',
 "Some of Bin Laden's close comrades were more peers than subordinates.",
 'For example, Usama Asmurai, also known as Wali Khan, worked with Bin Laden in the early 1980s and helped him in the Philippines and in Taji

In [16]:
question

{'question': "Who were considered Bin Laden's peers?",
 'sentences_used': [9, 11, 7, 8],
 'answers': [{'text': 'The Egyptian spiritual guide',
   'isAnswer': True,
   'scores': {}},
  {'text': 'Usama Asmurai', 'isAnswer': True, 'scores': {}},
  {'text': 'Wali Khan', 'isAnswer': True, 'scores': {}},
  {'text': 'the Blind Sheikh', 'isAnswer': True, 'scores': {}},
  {'text': 'Abu Zubaydah', 'isAnswer': False, 'scores': {}},
  {'text': 'Deaf Sheik', 'isAnswer': False, 'scores': {}},
  {'text': 'US Troops', 'isAnswer': False, 'scores': {}}],
 'idx': '11',
 'multisent': False}

In [7]:
def multirc_make_tests(question_text, answer_texts):
  tests = []
  for answer in answer_texts:
    pos = f'{question_text.strip()} {answer.strip()}'
    negs = [ f'{answer.strip()}', f'{question_text.strip()}' ]
    tests.append((pos, negs))
  return tests

In [27]:
datum = data[0]['paragraph']

In [25]:
datum['questions'][0]['answers'][0]

{'text': 'Deborah Russel and the media who saw the video as sexist',
 'isAnswer': True,
 'scores': {}}

In [32]:
multirc_make_test(datum['questions'][0]['question'], [ d['text'] for d in datum['questions'][0]['answers'] ])

[("Who were the people that opposed Air New Zealand's decisions? Deborah Russel and the media who saw the video as sexist",
  ['Deborah Russel and the media who saw the video as sexist',
   "Who were the people that opposed Air New Zealand's decisions?"]),
 ("Who were the people that opposed Air New Zealand's decisions? Sarah Gomez",
  ['Sarah Gomez',
   "Who were the people that opposed Air New Zealand's decisions?"]),
 ("Who were the people that opposed Air New Zealand's decisions? Sports Illustrated",
  ['Sports Illustrated',
   "Who were the people that opposed Air New Zealand's decisions?"]),
 ("Who were the people that opposed Air New Zealand's decisions? Deborah Russell",
  ['Deborah Russell',
   "Who were the people that opposed Air New Zealand's decisions?"]),
 ("Who were the people that opposed Air New Zealand's decisions? Christy Brinkley",
  ['Christy Brinkley',
   "Who were the people that opposed Air New Zealand's decisions?"])]

In [31]:
datum['questions'][0].keys()

dict_keys(['question', 'sentences_used', 'answers', 'idx', 'multisent'])

In [8]:
def get_pad_lens(m, pad=0):
  if len(m) < 2:
    return [0]*len(m)
  max_len = float('-inf')
  for row in m:
    max_len = max(max_len, len(row))
  pad_lens = []
  for row in m:
    pad_len = max_len-len(row)
    pad_lens.append(pad_len)
  return pad_lens

In [34]:
def get_lens(m):
  lens = []
  for row in m:
    lens.append(len(row))
  return lense